# 3 - Modelos Regulares

Este notebook implementa uma *terceira versão* de um modelo preditivo para estimar a **demanda horária** do pronto-socorro.

Os principais objetivos desta terceira versão são:
- Explorar modelos lineares regularizados, que são muito úteis quando há multicolinearidade (algo comum com lags e janelas móveis).
- Entender o papel da regularização na suavização e estabilidade das previsões.
- Comparar com o baseline linear para medir ganhos reais.

Seguiremos avaliando estas etapas:
1. Seleção das features e preparação dos dados
2. Divisão treino/teste respeitando series temporais
3. Treinamento do modelo baseline (Regressão Linear)
4. Predição
5. Avaliação com MAE, RMSE, MAPE e R²
6. Gráficos de diagnóstico

---

# 1. Importar bibliotecas e configurações iniciais

### 1.0 Instalações

In [105]:
# ! pip install scikit-learn

### 1.1 Importações

In [106]:
import pandas as pd
import numpy as np
import altair as alt

In [107]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error, mean_squared_error

import locale

- Nesta etapa utilizaremos modelos lineares regularizados, que aplicam penalidades aos coeficientes para controlar a variância do modelo, reduzir overfitting e lidar com multicolinearidade.

In [108]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [109]:
import sys
sys.path.append("../src")

from features.feature_engineering import create_lag_features, create_rolling_features, add_time_features
from training.model_evaluation import evaluate_model

### 1.2 Configurações de bibliotecas

In [110]:
# Desabilitar a limitação de linhas em gráficos do Altair
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [111]:
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

'pt_BR.UTF-8'

# 2. Importar e tratar dados

### 2.1 Importando dados

In [112]:
CAMINHO_DADOS = '../data/raw/dataset_pronto_socorro.csv'

df = pd.read_csv(CAMINHO_DADOS)

In [113]:
df.head()

,datetime,day_of_week,month,is_weekend,temperature,rain_mm,demand
0,2023-01-01 00:00:00,6,1,1,24.483571,0.353269,29.0
1,2023-01-01 01:00:00,6,1,1,21.308678,5.847757,30.0
2,2023-01-01 02:00:00,6,1,1,25.238443,1.141991,30.0
3,2023-01-01 03:00:00,6,1,1,29.615149,0.524987,33.0
4,2023-01-01 04:00:00,6,1,1,20.829233,0.820610,33.0


### 2.2 Tratamento de Dados

In [114]:
df.drop(columns=['day_of_week'], inplace=True)

In [115]:
# Converter coluna de data/hora para datetime e definir índice
if 'datetime' in df.columns:
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.set_index('datetime')

### 2.3 Engenharia de Featurees

In [116]:
df = df.copy()

# Adicionar variáveis temporais
df = add_time_features(df)

# Lags essenciais: 1 hora, 24h, 48h, 1 semana (168h)
df = create_lag_features(df, lags=[1, 2, 3, 24, 48, 168])

# Rolling windows
df = create_rolling_features(df, windows=[3, 6, 12, 24])

# Remover linhas com NaNs causados pelos lags/rolling
df = df.dropna()

In [117]:
df.head()

,month,is_weekend,temperature,rain_mm,demand,hour,dayofweek,demand_lag_1,demand_lag_2,demand_lag_3,...,demand_lag_48,demand_lag_168,demand_roll_mean_3,demand_roll_std_3,demand_roll_mean_6,demand_roll_std_6,demand_roll_mean_12,demand_roll_std_12,demand_roll_mean_24,demand_roll_std_24
datetime,,,,,,,,,,,,,,,,,,,,,
2023-01-08 00:00:00,1,1,20.773059,2.344874,25.0,0,6,28.0,26.0,20.0,...,22.0,29.0,26.333333,1.527525,24.166667,3.060501,24.083333,3.704011,25.583333,3.877658
2023-01-08 01:00:00,1,1,18.231319,0.268283,25.0,1,6,25.0,28.0,26.0,...,27.0,30.0,26.000000,1.732051,24.833333,2.639444,23.833333,3.511885,25.375000,3.762660
2023-01-08 02:00:00,1,1,17.552428,0.192845,22.0,2,6,25.0,25.0,28.0,...,24.0,30.0,24.000000,1.732051,24.333333,2.875181,23.583333,3.528026,25.000000,3.623594
2023-01-08 03:00:00,1,1,17.920949,1.191458,34.0,3,6,22.0,25.0,25.0,...,26.0,33.0,27.000000,6.244998,26.666667,4.082483,24.000000,4.410731,25.250000,4.024382
2023-01-08 04:00:00,1,1,21.614491,3.771095,35.0,4,6,34.0,22.0,25.0,...,23.0,33.0,30.333333,7.234178,28.166667,5.269409,24.833333,5.441145,25.666667,4.488310


In [118]:
# Opcional: features sazonais contínuas (útil até pro baseline)

# Sinais "sin" e "cos" transformam hora em um ciclo contínuo, evitando que "23" esteja longe de "0".

# df['sin_hour'] = np.sin(2 * np.pi * df['hour'] / 24)
# df['cos_hour'] = np.cos(2 * np.pi * df['hour'] / 24)

# 3 Desenvolvimento de Modelo Preditivo

### 3.1 Seleção das variáveis do modelo

In [119]:
X = df.drop(columns=["demand"])
y = df["demand"]

### 3.2 Divisão dos dados em treino/teste

In [120]:
# Dividindo os dados em treino e teste, a partir da função TimseSeriesSplit, que divide os dados 
tscv = TimeSeriesSplit(n_splits=10)

### 3.3 Criação de Pipelines de treino

#### 3.3.1 Ridge

In [121]:
ridge = Pipeline([
    ("scaler", StandardScaler()),
    ("model", Ridge(alpha=1.0))
])


#### 3.3.2 Lasso

In [122]:
lasso = Pipeline([
    ("scaler", StandardScaler()),
    ("model", Lasso(alpha=0.1))
])


#### 3.3.3 ElasticNet

In [123]:
elastic = Pipeline([
    ("scaler", StandardScaler()),
    ("model", ElasticNet(alpha=0.1, l1_ratio=0.5))
])


#### 3.3.4 LinearRegression

In [124]:
linear = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LinearRegression())
])


### 3.4 Validação dos modelos

In [125]:
results_ridge,df_ridge = evaluate_model(ridge, X, y, tscv)
results_lasso,df_lasso = evaluate_model(lasso, X, y, tscv)
results_enet,df_enet = evaluate_model(elastic, X, y, tscv)
results_linear,df_linear = evaluate_model(linear, X, y, tscv)

In [126]:
summary = pd.DataFrame({
    "Ridge": results_ridge.mean(),
    "Lasso": results_lasso.mean(),
    "ElasticNet": results_enet.mean(),
    "LinearRegression": results_linear.mean()
})
summary


,Ridge,Lasso,ElasticNet,LinearRegression
MAE,0.010797,0.591231,1.416766,1.361690e-14
RMSE,0.013497,0.738646,1.768722,1.610171e-14
MAPE,0.054530,3.111577,7.455637,7.004566e-14
R2,0.999986,0.972107,0.840376,1.000000e+00


In [127]:
summary_df = pd.concat([df_ridge,df_lasso,df_enet,df_linear],axis=0)
summary_df.head()

,model,y_true,y_pred
datetime,,,
2024-10-26 08:00:00,Ridge(),22.0,21.999562
2024-10-26 09:00:00,Ridge(),20.0,20.003488
2024-10-26 10:00:00,Ridge(),21.0,20.999176
2024-10-26 11:00:00,Ridge(),22.0,21.996015
2024-10-26 12:00:00,Ridge(),19.0,19.002475


# 4. Análise detalhada do resultado

### 4.1 Criar dataframe com resultados

In [128]:
# Erro residual
# Residual positivo = modelo subestimou
# Residual negativo = modelo superestimou

summary_df["residual"] = summary_df["y_true"] - summary_df["y_pred"]

In [129]:
summary_df.head()

,model,y_true,y_pred,residual
datetime,,,,
2024-10-26 08:00:00,Ridge(),22.0,21.999562,0.000438
2024-10-26 09:00:00,Ridge(),20.0,20.003488,-0.003488
2024-10-26 10:00:00,Ridge(),21.0,20.999176,0.000824
2024-10-26 11:00:00,Ridge(),22.0,21.996015,0.003985
2024-10-26 12:00:00,Ridge(),19.0,19.002475,-0.002475


## 4.3 Distribuição dos Resíduos

In [130]:
# Se os resíduos não forem centrados em 0 → viés no modelo.
# Caudas pesadas → eventos extremos não capturados.

In [131]:
# Lista de modelos existentes
modelos = summary_df["model"].unique()

charts = []

for m in modelos:
    df_m = summary_df[summary_df["model"] == m].reset_index()

    # Histograma
    hist = (
        alt.Chart(df_m)
        .mark_bar(opacity=0.6)
        .encode(
            x=alt.X("residual:Q", bin=alt.Bin(maxbins=50), title="Resíduos"),
            y=alt.Y("count()", title="Frequência"),
        )
    )

    # Linha vertical em zero
    linha_zero = (
        alt.Chart(pd.DataFrame({"x": [0]}))
        .mark_rule(color="black")
        .encode(x="x:Q")
    )

    # KDE
    kde = (
        alt.Chart(df_m)
        .transform_density(
            "residual",
            as_=["residual", "density"],
        )
        .mark_line(color="red")
        .encode(
            x="residual:Q",
            y="density:Q",
        )
    )

    # Combine para esse modelo
    chart_m = (
        alt.layer(hist, linha_zero, kde)
        .resolve_scale(y="independent")
        .properties(
            width=500,
            height=400,
            title=f"Modelo: {m}"
        )
    )

    charts.append(chart_m)

# Concat horizontal
alt.hconcat(*charts)


alt.HConcatChart(...)

### 4.4 Error por Hora do dia

- Este gráfico revela se o modelo falha no começo, meio ou final do dia.
- Se os erros forem sistematicamente altos em certas horas → falta de features temporais.

In [132]:
df_hour = summary_df.copy()
df_hour["hour"] = summary_df.index.hour

hour_mae = df_hour.groupby(by=['model',"hour"],as_index=False)["residual"].apply(lambda x: np.mean(np.abs(x)))

In [133]:
charts = []

for m in modelos:
    df_m = hour_mae[hour_mae["model"] == m].reset_index()

    chart = (
        alt.Chart(df_m)
        .mark_bar()
        .encode(
            x=alt.X("hour:O", title="Hora do dia"),
            y=alt.Y("residual:Q", title="Erro absoluto médio (MAE)"),
        )
        .properties(
            width=600,
            height=300,
            title=f"MAE por hora do dia (Modelo: {m})"
        )
    )

    charts.append(chart)

# Concat horizontal
alt.vconcat(*charts)

alt.VConcatChart(...)

### 4.5 Erro por dia da Semana

- A demanda hospitalar costuma variar muito entre domingo e segunda.
- Se o baseline não captura → será necessário adicionar lags.

In [134]:
df_dow = summary_df.copy()
df_dow["dayofweek"] = summary_df.index.dayofweek

dow_mae = df_dow.groupby(['model',"dayofweek"],as_index=False)["residual"].apply(lambda x: np.mean(np.abs(x)))
dow_mae.head()

,model,dayofweek,residual
0,ElasticNet(alpha=0.1),0,1.537384
1,ElasticNet(alpha=0.1),1,1.406582
2,ElasticNet(alpha=0.1),2,1.557785
3,ElasticNet(alpha=0.1),3,1.447305
4,ElasticNet(alpha=0.1),4,1.460020


In [135]:
charts = []

for m in modelos:
    df_m = dow_mae[dow_mae["model"] == m].reset_index()

    chart = (
        alt.Chart(df_m)
        .mark_bar()
        .encode(
            x=alt.X("dayofweek:O", title="Dia da Semana"),
            y=alt.Y("residual:Q", title="Erro absoluto médio (MAE)"),
        )
        .properties(
            width=600,
            height=300,
            title=f"MAE por dia da semana (Modelo: {m})"
        )
    )

    charts.append(chart)

# Concat horizontal
alt.vconcat(*charts)

alt.VConcatChart(...)

### 4.6 Comparação Real vs. Predição agregado por dia

- Linear Regression pode acertar nível médio mas errar amplitude.
- Se o modelo suaviza demais → pode exigir modelos não lineares.

In [136]:
df_hour = summary_df.copy().reset_index()
df_hour["date"] = summary_df.index.date

daily_error = df_hour.groupby(by=['model',"date"],as_index=False)[['y_true',"y_pred"]].sum()
daily_error.head()

,model,date,y_true,y_pred
0,ElasticNet(alpha=0.1),2024-10-26,313.0,316.889773
1,ElasticNet(alpha=0.1),2024-10-27,515.0,521.218897
2,ElasticNet(alpha=0.1),2024-10-28,397.0,399.438202
3,ElasticNet(alpha=0.1),2024-10-29,398.0,400.643315
4,ElasticNet(alpha=0.1),2024-10-30,378.0,382.752594


In [137]:
charts = []

for m in modelos:
    df_m = daily_error[daily_error["model"] == m].reset_index()

    # # Converte para formato long (necessário para múltiplas linhas no Altair)
    df_long = df_m.melt(id_vars=["date"], value_vars=["y_true", "y_pred"],
                           var_name="tipo", value_name="valor")
    df_long['date'] = pd.to_datetime(df_long['date'])

    chart = (
        alt.Chart(df_long)
        .mark_line()
        .encode(
            x=alt.X("date:T", title="Data"),
            y=alt.Y("valor:Q", title="Demanda"),
            color=alt.Color("tipo:N", title="Série", scale=alt.Scale(
                domain=["y_true", "y_pred"],
                range=["black", "steelblue"]
            )),
            tooltip=["date:T", "tipo:N", "valor:Q"]
        )
        .properties(
            width=1200,
            height=500,
            title=f"Demanda diária — Real vs Prevista (Modelo: {m})"
        )
    )

    charts.append(chart)

alt.vconcat(*charts)


# df_long

alt.VConcatChart(...)

### 4.7 Resíduos ao longo de tempo

- Ver se há períodos em que o modelo erra sistematicamente (viés temporal).
- Ver se há heterocedasticidade (erro aumenta em períodos de pico).

In [138]:
charts = []

for m in modelos:
    df_m = summary_df[summary_df["model"] == m].reset_index()

    # Linha dos resíduos
    residual_line = (
        alt.Chart(df_m.reset_index())
        .mark_line()
        .encode(
            x=alt.X("datetime:T", title="Data"),
            y=alt.Y("residual:Q", title="Resíduo")
        )
    )

    # Linha horizontal em zero
    linha_zero = (
        alt.Chart(pd.DataFrame({"y": [0]}))
        .mark_rule(color="black")
        .encode(y="y:Q")
    )

    chart = (
        (residual_line + linha_zero)
        .properties(
            width=1200,
            height=400,
            title="Resíduos ao longo do tempo"
        )
    )

    charts.append(chart)

# Concat horizontal
alt.vconcat(*charts)

alt.VConcatChart(...)

### 4.8 Interpretação dos coeficientes do modelo

- coef > 0 → aumenta demanda
- coef < 0 → reduz demanda
  
- Permite validar coerência com conhecimento hospitalar.

In [139]:
# coef_df = pd.DataFrame({
#     "feature": features,
#     "coef": model.coef_
# })

# coef_df.sort_values("coef", ascending=False)

### 4.9 Importância padronizada (coef*std)

- Isso mostra quais variáveis mais impactam a previsão na prática.
- Muito útil para justificar a evolução do modelo.

In [140]:
# stds = X_train.std()

# coef_imp_df = pd.DataFrame({
#     "feature": features,
#     "coef": model.coef_,
#     "std": stds,
# })

# coef_imp_df["importance_std"] = coef_imp_df["coef"] * coef_imp_df["std"]
# coef_imp_df.sort_values("importance_std", ascending=False)

# 5. Conclusão

- Implementamos uma pipeline de treinamento, a qual normaliza todas as variáveis para garantir que todas estejam na mesma escala, reduzindo seu impacto na modelagem;
- Mesmo utilizando outros três modelos, o Ridge, Lasso e ElasticNet, a Regressão Linear manteve seu excelente resultado em todas as métricas;

**Próximos passos**
- Finalizar o projeto salvando o modelo em um formato reutilizável e fazer previsões futaras.